# 3D dose map analysis

Use the output of the following simulation:
- Folder: dosimetry/
- Macros: ex2.mac, ex3.mac

Helping ressources: http://insightsoftwareconsortium.github.io/SimpleITK-Notebooks

In [116]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import SimpleITK as sitk
from ipywidgets import interact
import os
from pathlib import Path

In order to be able to run the interactive plot, you must install ipympl as indicated here: https://github.com/matplotlib/ipympl
This is activated by the previous `%matplotlib widget` line.

In [117]:
# The following command display the current working directory (where jupyter has been launched)
cwd = os.getcwd()
print('The Current Working Directory (CWD) is: \n', cwd)
folder = Path()
if (not folder.is_dir()):
    print('ERROR: {} is not a folder.'.format(folder))

The Current Working Directory (CWD) is: 
 /Users/dsarrut/src/gate/dqprm/2021/gate-exercices/dosimetry


In [118]:
# Read a sitk image
filename = os.path.join(folder,'./data/patient-2mm.mhd')
img_ct = sitk.ReadImage(filename)
print('Image size: ', img_ct.GetSize())
print('Image spacing: ', img_ct.GetSpacing())
print('Image origin: ', img_ct.GetOrigin())

Image size:  (126, 126, 111)
Image spacing:  (2.0, 2.0, 2.0)
Image origin:  (0.0, 0.0, 0.0)


In [119]:
# Convert sitk image to a numpy array
arr_ct = sitk.GetArrayFromImage(img_ct)
print('Array size: ', arr_ct.shape, ' <--- be careful to the dimension order!)')

Array size:  (111, 126, 126)  <--- be careful to the dimension order!)


In [120]:
fig, ax = plt.subplots(1,3, figsize=(8, 2))
def show_ct(sx,sy,sz):
    ax[0].imshow(arr_ct[sz,:,:], cmap=plt.cm.gray)
    ax[1].imshow(arr_ct[:,sx,:], cmap=plt.cm.gray)
    ax[2].imshow(arr_ct[:,:,sy], cmap=plt.cm.gray)
    
interact(show_ct, sx=(0,img_ct.GetSize()[0]), sy=(0,img_ct.GetSize()[1]), sz=(0,img_ct.GetSize()[2]));

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=63, description='sx', max=126), IntSlider(value=63, description='sy', ma…

In [121]:
filename = os.path.join(folder, './output_ref/3d-pat-proton-Dose.mhd')    # <--------------------------------------TO CHANGE BY YOUR OWN OUTPUT FOLDER 
img_dose = sitk.ReadImage(filename)
arr_dose = sitk.GetArrayFromImage(img_dose)
print('Image size = ', arr_dose.shape)
print('Image min and max: ',  np.amin(arr_dose), np.amax(arr_dose))

Image size =  (50, 50, 50)
Image min and max:  0.0 2.5758645e-05


In [122]:
filter = sitk.RescaleIntensityImageFilter()
filter.SetOutputMaximum(1.0)
filter.SetOutputMinimum(0.0)
img_dose = filter.Execute(img_dose)

In [123]:
arr_dose = sitk.GetArrayFromImage(img_dose)
print('Image size = ', arr_dose.shape)
print('Image min and max: ',  np.amin(arr_dose), np.amax(arr_dose))

Image size =  (50, 50, 50)
Image min and max:  0.0 1.0


In [124]:
fig1, ax1 = plt.subplots()
def show_dose(nslice):
    im = ax1.imshow(arr_dose[:,nslice,:], cmap=plt.cm.gray)
interact(show_dose, nslice=(0,len(arr_dose)-1));

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=24, description='nslice', max=49), Output()), _dom_classes=('widget-inte…

In [125]:
img_resampled_dose = sitk.Resample(img_dose, img_ct, sitk.Transform(), sitk.sitkLinear, 0)

In [126]:
arr_resampled_dose = sitk.GetArrayFromImage(img_resampled_dose)
print('Image size = ', arr_resampled_dose.shape)
print('Image min and max: ',  np.amin(arr_resampled_dose), np.amax(arr_resampled_dose))

Image size =  (111, 126, 126)
Image min and max:  0.0 0.94772357


In [127]:
fig2, ax2 = plt.subplots()
def show_fusion(nslice=58, opacity=0.7):
    ax2.imshow(arr_ct[:,:,nslice], cmap=plt.cm.gray)
    a = arr_resampled_dose[:,:,nslice]
    b = np.ma.masked_where(a <= 0.001, a)
    ax2.imshow(b, alpha=opacity, cmap=plt.cm.hot)
    
interact(show_fusion, nslice=(0,len(arr_ct)), opacity=(0,1,0.1));

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=58, description='nslice', max=111), FloatSlider(value=0.7, description='…

# Part 2: 1D plot from 3D proton dose

In [128]:
filename_edep = os.path.join(str(folder), './output_ref/3d-pat-proton-Edep.mhd')
img_edep = sitk.ReadImage(filename_edep)
arr_edep = sitk.GetArrayFromImage(img_edep)

In [129]:
# Declare a single figure (one row, one column)
fig3, ax3 = plt.subplots()
moy_dose=np.zeros(arr_dose.shape[0])
moy_edep=np.zeros(arr_edep.shape[0])
for i in range(arr_dose.shape[0]):
    subarr_dose=arr_dose[i,:,:]
    moy_dose[i]=np.mean(subarr_dose)
    subarr_edep=arr_edep[i,:,:]
    moy_edep[i]=np.mean(subarr_edep)

# X values from 0 to n
# n is the number of slices
n = arr_dose.shape[0]
x = np.linspace(0, n, n)

c1 = ax3.plot(x, moy_dose, 'k.-', label='deposited dose', linewidth=1)
ax32 = ax3.twinx()
c2 = ax32.plot(x, moy_edep, 'b.-', label='deposited energy', linewidth=1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Other examples of exercices

- Plot a depth dose profile with interactive line coordinate
- Display uncertainty map
- Create another simulation with a different beam angle
- Sum and display two different dose maps (two different irradiation fields for example)